In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json


In [ ]:

def has_bad_attrs(element):
  bad_ids = ['app-promotion', 'ad-intersect']
  for id in bad_ids:
    if ('id' in element.attrs and id in element['id']):
      return True

  bad_classes = ['serp-banner', 'serp-teads', 'advergic']
  for cl in bad_classes:
    if (cl in element['class']):
      return True
  return False

def get_dubicars_page(page):
  data = requests.get('https://www.dubicars.com/search?o=&did=&gen=&ul=RU&ma=13&mo=0&b=&set=bu&eo=export-only&stsd=&cr=USD&cy=&co=&s=&gi=&f=&g=&l=&st=&page='+str(page))
  soup = BeautifulSoup(data.content)
  main_elements = soup.find_all("div", {"class" : "main"})
  cars = main_elements[0].find_all(recursive=False)[0].find_all(recursive=False)[0].find_all(recursive=False)



  cars = [car for car in cars if
        not has_bad_attrs(car)]



  cars_jsoned = [json.loads(car['data-ga4-detail']) for car in cars]

  data_frame_with_cars = pd.DataFrame(index=[car_param_name for car_param_name in cars_jsoned[0].keys()])
  for car in cars_jsoned:
    data_frame_with_cars[car['car_make'] + '_' + car['car_model'] + '_' + str(car['item_id'])] = [car_param for car_param in car.values()]

  print(f'[log]: processed page {i}')

  return data_frame_with_cars

In [ ]:
def get_total_pages():
  data = requests.get('https://www.dubicars.com/search?o=&did=&gen=&ul=RU&ma=13&mo=0&b=&set=bu&eo=export-only&stsd=&cr=USD&cy=&co=&s=&gi=&f=&g=&l=&st=&page=1')
  soup = BeautifulSoup(data.content)
  main_elements = soup.find_all("div", {"class" : "main"})
  return int(main_elements[1].find_all("li", {"class" : "base-btn"})[-1].find_all()[0].get_text())

def get_all_cars():
  total_pages = get_total_pages()
  return pd.concat([get_dubicars_page(i) for i in range(total_pages)], axis = 1)

all_cars = get_all_cars()

In [ ]:
def refactor_data(cars_data):
  new_data = cars_data.copy()
  new_data.loc['price'].apply(int)
  new_data.loc['price'] = new_data.loc['price']*23.4 #converting to rubbles #TODO add converter
  mask = (new_data.loc['price'] > 0);
  display(new_data.loc[:, mask])
  new_data = new_data.loc[:, mask]

  return new_data
all_cars_dataset = refactor_data(all_cars)

,BMW_X7_831634,BMW_X1_831349,BMW_X6_709578,BMW_X5_740792,BMW_X6_836233,BMW_1M_836052,BMW_118i_832952,BMW_228i_818492,BMW_X5_696769,BMW_X6_832171,...,BMW_528i_717456,BMW_740Li_739011,BMW_X4_730256,BMW_730Li_648796,BMW_430i_664338,BMW_730Li_705068,BMW_X5M_666241,BMW_ActiveHybrid 7_592012,BMW_530i_567860,BMW_330i_779371
item_id,831634,831349,709578,740792,836233,836052,832952,818492,696769,832171,...,717456,739011,730256,648796,664338,705068,666241,592012,567860,779371
seller_type,Private,Private,Dealer,Dealer,Private,Private,Private,Private,Dealer,Dealer,...,Dealer,Dealer,Dealer,Dealer,Dealer,Dealer,Dealer,Dealer,Dealer,Dealer
price,7956000.0,1450800.0,3252600.0,3627000.0,5148000.0,3393000.0,421200.0,2386800.0,7956000.0,1521000.0,...,1380600.0,3978000.0,1497600.0,2223000.0,1848600.0,1006200.0,4188600.0,889200.0,538200.0,2410200.0
export_status,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,...,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported
export_price,92643,16893,38100,0,59945,39509,4904,27792,92643,17800,...,16212,46600,0,26200,21662,11852,48773,10354,6300,28200
feature_type,Premium,Premium,Premium,Premium,Premium,Premium,Premium,Premium,Premium,Premium,...,Basic,Basic,Basic,Basic,Basic,Basic,Basic,Basic,Basic,Basic
car_year,2024,2020,2019,2018,2021,2011,2012,2022,2024,2015,...,2016,2022,2015,2017,2017,2014,2019,2010,2006,2022
warranty,0,0,1,1,0,0,0,0,1,0,...,1,0,1,1,1,0,1,0,0,0
steering_side,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,...,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand
promoted,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
all_cars_dataset

,BMW_X7_831634,BMW_X1_831349,BMW_X6_709578,BMW_X5_740792,BMW_X6_836233,BMW_1M_836052,BMW_118i_832952,BMW_228i_818492,BMW_X5_696769,BMW_X6_832171,...,BMW_528i_717456,BMW_740Li_739011,BMW_X4_730256,BMW_730Li_648796,BMW_430i_664338,BMW_730Li_705068,BMW_X5M_666241,BMW_ActiveHybrid 7_592012,BMW_530i_567860,BMW_330i_779371
item_id,831634,831349,709578,740792,836233,836052,832952,818492,696769,832171,...,717456,739011,730256,648796,664338,705068,666241,592012,567860,779371
seller_type,Private,Private,Dealer,Dealer,Private,Private,Private,Private,Dealer,Dealer,...,Dealer,Dealer,Dealer,Dealer,Dealer,Dealer,Dealer,Dealer,Dealer,Dealer
price,7956000.0,1450800.0,3252600.0,3627000.0,5148000.0,3393000.0,421200.0,2386800.0,7956000.0,1521000.0,...,1380600.0,3978000.0,1497600.0,2223000.0,1848600.0,1006200.0,4188600.0,889200.0,538200.0,2410200.0
export_status,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,...,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported,can_be_exported
export_price,92643,16893,38100,0,59945,39509,4904,27792,92643,17800,...,16212,46600,0,26200,21662,11852,48773,10354,6300,28200
feature_type,Premium,Premium,Premium,Premium,Premium,Premium,Premium,Premium,Premium,Premium,...,Basic,Basic,Basic,Basic,Basic,Basic,Basic,Basic,Basic,Basic
car_year,2024,2020,2019,2018,2021,2011,2012,2022,2024,2015,...,2016,2022,2015,2017,2017,2014,2019,2010,2006,2022
warranty,0,0,1,1,0,0,0,0,1,0,...,1,0,1,1,1,0,1,0,0,0
steering_side,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,...,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand,Left hand
promoted,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
import plotly.express as px
fig = px.bar(x=all_cars_dataset.columns, y=all_cars_dataset.loc['price'])
fig.update_layout( xaxis={'categoryorder':'total ascending'})

fig.show()

In [ ]:
import numbers
all_cars_dataset_2 = all_cars_dataset.copy()
all_cars_dataset_2 = all_cars_dataset_2.transpose()
grouped = all_cars_dataset_2.groupby("car_model")
display(grouped["price"].agg(["sum", "mean", "std"]))

all_cars_dataset_2 = grouped.apply(lambda x: x).transpose()

display(all_cars_dataset_2)


prices = grouped["price"].agg(["sum", "mean", "std"]);

fig = px.bar(x=prices.index, y=prices['mean'])
fig.update_layout( xaxis={'categoryorder':'total ascending'})

fig.show()

,sum,mean,std
car_model,,,
118i,17877600.0,2979600.0,1.987136e+06
120i,17994600.0,2999100.0,7.511971e+05
1M,12846600.0,4282200.0,1.540140e+06
2002,1965600.0,1965600.0,NaN
218i,10108800.0,3369600.0,5.850000e+05
...,...,...,...
i7,77454000.0,15490800.0,2.042647e+06
i8,16450200.0,5483400.0,1.454820e+06
iX,55294200.0,6911775.0,1.905872e+06


<ipython-input-146-f05672acbb74>:7: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



car_model                                                    118i  \
                                                  BMW_118i_832952   
item_id                                                    832952   
seller_type                                               Private   
price                                                    421200.0   
export_status                                     can_be_exported   
export_price                                                 4904   
feature_type                                              Premium   
car_year                                                     2012   
warranty                                                        0   
steering_side                                           Left hand   
promoted                                                    False   
dealer_site                                                 False   
image_url       https://dubicars.com/images/4e940a/w_1300x760/...   
car_make                                                      BMW   
car_model                                                    118i   
body_type                                               Hatchback   
condition                                                    Used   
fuel_type                                                Gasoline   
regional_specs                                                GCC   
seller_name                                        Private Seller   

car_model                                                          \
                                                  BMW_118i_830288   
item_id                                                    830288   
seller_type                                                Dealer   
price                                                   4095000.0   
export_status                                     can_be_exported   
export_price                                                47900   
feature_type                                              Premium   
car_year                                                     2025   
warranty                                                        0   
steering_side                                           Left hand   
promoted                                                    False   
dealer_site                                                 False   
image_url       https://dubicars.com/images/4f1ac7/w_1300x760/...   
car_make                                                      BMW   
car_model                                                    118i   
body_type                                               Hatchback   
condition                                                    Used   
fuel_type                                                Gasoline   
regional_specs                                                GCC   
seller_name                                Abu Dhabi Motors - BMW   

car_model                                                          \
                                                  BMW_118i_832952   
item_id                                                    832952   
seller_type                                               Private   
price                                                    421200.0   
export_status                                     can_be_exported   
export_price                                                 4904   
feature_type                                              Premium   
car_year                                                     2012   
warranty                                                        0   
steering_side                                           Left hand   
promoted                                                    False   
dealer_site                                                 False   
image_url       https://dubicars.com/images/4e940a/w_1300x760/...   
car_make                                                      BMW   
car_model                                                    118i